In [2]:
import pandas as pd
import numpy as np
# --------- for visualization ---------
import matplotlib.pyplot as plt
import seaborn as srn

from sklearn.model_selection import train_test_split
from sklearn.metrics import*
from keras.utils import to_categorical

# --------- for DL model ---------
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

# **Load datasets (Train & Test)**

In [3]:
dataTrainDF =  pd.read_csv("data/train.csv")
dataTestDF =  pd.read_csv("data/test.csv")

xTrain, yTrain = dataTrainDF[dataTrainDF.columns[1:]].values, dataTrainDF["label"].values
xTest = dataTestDF.values

# **Constructing the model!**

In [4]:
activationFunction='relu'
batch_size = 32
epoch = 100
numOfChannels = 1
img_rows, img_cols = 28, 28
numClasses = 10
def getModel():
    model = Sequential()
    
    model.add(Conv2D(100, kernel_size = (3, 3), activation=activationFunction, input_shape=(img_rows, img_cols, numOfChannels)))
    model.add(Dropout(0.1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Conv2D(50, kernel_size=(3,3), activation=activationFunction))
    model.add(Dropout(0.1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Conv2D(25, kernel_size=(3,3), activation=activationFunction))
    model.add(Dropout(0.1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Flatten())
    model.add(Dense(100, activation=activationFunction))
    model.add(Dropout(0.1))
    model.add(Dense(50, activation=activationFunction))
    model.add(Dense(numClasses, activation = 'softmax'))
    model.compile(optimizer='RMSprop', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

model = getModel()

# **Pre-processing dataset**

In [5]:
xTrain = np.reshape(xTrain,(-1, img_rows, img_cols, 1))
xTest = np.reshape(xTest,(-1, img_rows, img_cols, 1))

xTrain = xTrain.astype('float32')
xTest = xTest.astype('float32')
xTrain = xTrain / 255.
xTest = xTest / 255.

#One hot encoding
yTrain0HE = np.array(to_categorical(yTrain))

In [6]:
print("xTrain", xTrain.shape, "yTrain0HE", yTrain0HE.shape)
print("xTest", xTest.shape)

xTrain (42000, 28, 28, 1) yTrain0HE (42000, 10)
xTest (28000, 28, 28, 1)


# **Learning rate scheduler**

In [7]:
def adapt_learning_rate(epoch):
    return 0.001 / (epoch+1)
lrScheduler = keras.callbacks.LearningRateScheduler(adapt_learning_rate)

# **Train the model**

In [8]:
training = model.fit(xTrain, yTrain0HE, batch_size=batch_size, epochs=epoch, verbose=1, callbacks=[lrScheduler])
model.save("CNN_model.h5")

Epoch 1/100
1313/1313 [==============================] - 6s 5ms/step - loss: 0.3635 - accuracy: 0.8830
Epoch 2/100
1313/1313 [==============================] - 6s 5ms/step - loss: 0.1039 - accuracy: 0.9688
Epoch 3/100
1313/1313 [==============================] - 6s 5ms/step - loss: 0.0768 - accuracy: 0.9765
Epoch 4/100
1313/1313 [==============================] - 6s 5ms/step - loss: 0.0646 - accuracy: 0.9802
Epoch 5/100
1313/1313 [==============================] - 6s 5ms/step - loss: 0.0565 - accuracy: 0.9825
Epoch 6/100
1313/1313 [==============================] - 6s 5ms/step - loss: 0.0518 - accuracy: 0.9835
Epoch 7/100
1313/1313 [==============================] - 6s 5ms/step - loss: 0.0497 - accuracy: 0.9851
Epoch 8/100
1313/1313 [==============================] - 6s 5ms/step - loss: 0.0450 - accuracy: 0.9863
Epoch 9/100
1313/1313 [==============================] - 6s 5ms/step - loss: 0.0421 - accuracy: 0.9870
Epoch 10/100
1313/1313 [==============================] - 6s 5ms/step - l

# **Predicting the test data**

In [9]:
yPredictProb = model.predict(xTest)
yPredictProb

array([[2.2720185e-05, 1.3097300e-08, 9.9996006e-01, ..., 1.5325819e-05,
        1.0852547e-06, 9.5323202e-08],
       [9.9997699e-01, 2.8585472e-08, 6.7112023e-07, ..., 2.6122524e-07,
        7.4554627e-08, 3.8853036e-06],
       [1.7609118e-08, 1.9374777e-10, 3.2036510e-08, ..., 1.4965345e-06,
        1.7252258e-05, 9.9991977e-01],
       ...,
       [1.0932775e-14, 1.0603851e-10, 1.1699335e-09, ..., 7.8496498e-10,
        2.0567142e-10, 2.8900832e-10],
       [4.4025974e-08, 3.4830364e-10, 6.8776771e-09, ..., 8.6853237e-07,
        3.4330802e-07, 9.9996567e-01],
       [2.0696236e-06, 4.3159285e-09, 9.9999487e-01, ..., 9.1761956e-07,
        6.2790701e-07, 5.4659379e-09]], dtype=float32)

# **Saving the predictions!**

In [10]:
yPred = np.argmax(np.round(yPredictProb),axis=1) #Select the class with maximum probability
predictionDF = pd.DataFrame({'ImageId': np.arange(1,len(yPred)+1,1), 'Label': yPred})

predictionDF.to_csv("result/submissionDL.csv",index=False)
predictionDF.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
